# 🧪 <span style="color:#6f5b85;">  TP: Simulating and modeling material properties at the atomic scale <span>

Welcome to this hands-on session, where we will explore atomic-scale physical phenomena such as *diffusion* and *plasticity* using **atomistic simulations**. 

We will work with the code **LAMMPS (https://docs.lammps.org/Manual.html)**, a widely used molecular dynamics engine in the materials science community, through the **Python interface provided by ASE** (Atomic Simulation Environment). ASE offers a flexible and user-friendly framework for building, running, and analyzing atomistic simulations in a programmable way.


🔧 **Tools we will use**:
- **LAMMPS**: classical molecular dynamics simulator
- **Python + ASE**: scripting and analysis interface

---

## 🗺️ <span style="color:#957bb1;"> Objectives <span>

In this session, we will investigate two fundamental phenomena in materials science, combining **molecular dynamics (MD)** with a simplified **kinetic Monte Carlo (KMC)** approach:

1. 🔬 **Point defect diffusion in a metal**  
   → From atomistic dynamics to kinetic modeling (simple KMC)

2. 🛠️ **Nanoindentation**  
   → Simulating the mechanical response of a material under local deformation

---

🎯 By the end of this lab, you will be able to:
- Run and control LAMMPS simulations via Python/ASE
- Analyze atomic-scale mechanisms
- Bridge the gap between atomistic MD and mesoscopic KMC modeling



## <span style="color:#957bb1;"> About LAMMPS code <span>

**LAMMPS** (Large-scale Atomic/Molecular Massively Parallel Simulator) is a powerful and widely-used molecular dynamics simulation package developed by Sandia National Laboratories.  
You can install LAMMPS from its official source code repository on GitHub:  

```bash
git clone https://github.com/lammps/lammps.git
cd lammps
mkdir build
cd build
cmake ../cmake
make -j4
sudo make install
```

### <span style="color:#7140ff">However, in this TP, we will use LAMMPS indirectly via the Python interface provided by **ASE** (Atomic Simulation Environment). ASE makes it easier to set up simulations, submit them to LAMMPS, and analyze results, all within Python.<span>



# <span style="color:#6f5b85;"> Example: Running a simple LAMMPS calculation via ASE <span>


## 1) Build the system

In the solid state, most metals adopt a **crystalline structure**, meaning that their atoms are arranged in a periodic and highly ordered lattice that extends in all three spatial dimensions. This long-range order results from the minimization of the system's free energy and reflects the balance between atomic bonding forces and packing efficiency.

The fundamental building block of a crystal is the **unit cell** — the smallest repeatable volume that, when translated through space, reconstructs the entire lattice. Its geometry and atomic content define the material’s symmetry and physical properties.

Common metallic crystal structures include:

- **Face-Centered Cubic (FCC)**: atoms at each cube corner and at the center of each face (e.g., Cu, Al, Ni, Au).
- **Body-Centered Cubic (BCC)**: atoms at each corner and one atom in the center of the cube (e.g., Fe, Mo, Cr).
- **Hexagonal Close-Packed (HCP)**: atoms arranged in a hexagonal pattern with dense atomic layers (e.g., Mg, Ti, Zn).

For this tutorial, we will focus on **aluminum**, a metal with an FCC crystal structure.

In [8]:
from ase.build import bulk

# 'Al' is the atom type, 'a' the lattice parameter  
unit_cell = bulk('Al', crystalstructure='fcc', a=4.05, cubic=True)


In [9]:
# it is possible to visalize the cell using nglview

import nglview as nv
# Visualisation interactive
view = nv.show_ase(unit_cell)
view.add_unitcell()
view.background = 'white'
view.camera = 'orthographic'
view

NGLWidget()

The unit cell can be replicated along each spatial direction to build a larger **supercell**, which is often useful to better capture the material's periodicity or to simulate extended systems.

For example, to replicate the unit cell three times in each direction, you can use:

```python
supercell = unit_cell.repeat((3, 3, 3))


In [10]:
super_cell = unit_cell.repeat((3, 3, 3))
view = nv.show_ase(super_cell)
view.add_unitcell()
view.background = 'white'
view.camera = 'orthographic'
view


NGLWidget()

### <span style="color:#60a976;"> To do : Try different crystal structures and visualize it. <span>

## 2) Choice of the interatomic potential 

In molecular dynamics simulations, the choice of **interatomic potential** is critical for accurately capturing the physical behavior of materials. For metals, several classes of **empirical potentials** have been developed to model metallic bonding and its many-body nature.

The most commonly used potentials for metals include:

- **EAM (Embedded Atom Method)**  
  The total energy is given by:  
  $$
  E = \sum_i F_i(\rho_i) + \frac{1}{2} \sum_{i \neq j} \phi_{ij}(r_{ij})
  $$
  where $F_i$ is the embedding function, $\rho_i$ is the local electron density, and $\phi_{ij}$ is a pair interaction.

- **MEAM (Modified Embedded Atom Method)**  
  Extends EAM to include angular dependence for more complex crystal structures.

- **Finnis–Sinclair (FS)**  
  A variant of EAM with a simpler form for the embedding function.

- **Tersoff or Stillinger-Weber**  
  Mainly used for covalent systems, but occasionally adapted for metal-semiconductor systems.

These potentials are not universal — they are fitted for specific elements or alloys and must be chosen carefully based on the system and properties of interest.

🗂️ **Where to find LAMMPS-compatible potentials**:
- [LAMMPS Potential Directory](https://www.lammps.org/patches/poems/potentials.html)
- [NIST Interatomic Potentials Repository](https://www.ctcms.nist.gov/potentials/)



For this tutorial, we will use the **Angelo et al. (1995)** EAM potential, available from the [NIST Interatomic Potentials Repository](https://www.ctcms.nist.gov/potentials/Download/1995--Angelo-J-E-Moody-N-R-Baskes-M-I--Ni-Al-H/1/NiAlH_jea.eam.alloy), which is designed for the **Ni–Al–H** system.

This potential is particularly suitable for modeling **Ni–Al alloys**, and is capable of capturing essential features of their **thermodynamic and mechanical behavior**, including phase stability and defect energetics, thanks to its formulation within the **Embedded Atom Method (EAM)** framework that accounts for the many-body nature of metallic bonding.

